## Evolučně generovaná strategie pro iterované vězňovo dilema

Vězňovo dilema převedené do podoby zápasu dvou hráčů usilujících o co největší bodový zisk v předem definovaném počtu kol. Součástí řešení je závěrečné porovnání jednotlivých strategií - zejména "best" strategie evolučně vygenerované pomocí knihovny DEAP - prostřednictvím turnaje každého s každým.

In [7]:
import numpy as np
import random
from deap import base, creator, tools, algorithms

### Typy hráčských strategií v podobě funkcí
Čtyři základní hráčské typy vracející v každém kole v podobě hodnoty 0/1 rozhodnutí, zda zradit či spolupracovat. Ke dvěma původně definovaným byla doplněná komplementární strategie "vždy zradit" a pomstychtivá "oko za oko".

In [8]:

# vždy kooperuje
def always_cooperate(myhistory, otherhistory):
    return 0


# náhodná odpověď
def random_answer(myhistory, otherhistory):
    p = random.random()
    if p < 0.5:
        return 1    
    return 0


# vždy zradí - DOPLNĚNO
def always_betray(myhistory, otherhistory):
    return 1


# oko za oko - DOPLNĚNO
def tit_for_tat(myhistory, otherhistory):
    if not otherhistory:
        return 0
    else:
        posledni_tah_protihrace = otherhistory[-1]
        return posledni_tah_protihrace


### Tabulka odměn a průběh hry
Pro vývoj strategií je klíčová matice určující bodovou odměnu (skóre) za každé kolo. Můžeme ji reprezentovat jako pole čtyř dvojic s variacemi hodnot 0 a 1 (spolupráce a zrada):
- (0, 0) - hráč A spolupracuje, hráč B spolupracuje
- (0, 1) - hráč A spolupracuje, hráč B zrazuje
- (1, 0) - hráč A zrazuje, hráč B spolupracuje
- (1, 1) - hráč A zrazuje, hráč B zrazuje

In [ ]:
# původní hodnocení rozdávající roky za mřížemi upraveno do praktičtější podoby zisku pozitivních bodů
def rozdej_skore(tah1, tah2):
    if (tah1 == 0) and (tah2 == 0):  # CC
        return (3, 3)
    elif (tah1 == 0) and (tah2 == 1):  # CD
        return (0, 5)
    elif (tah1 == 1) and (tah2 == 0):  # DC
        return (5, 0)
    elif (tah1 == 1) and (tah2 == 1):  # DD
        return (1, 1) 
        
        
def play(f1, f2, stepsnum):
    
    skore1 = 0
    skore2 = 0
    
    historie1 = []
    historie2 = []
    
    for i in range(stepsnum):
        tah1 = f1(historie1, historie2)
        tah2 = f2(historie2, historie1)

        s1, s2 = rozdej_skore(tah1, tah2)
        skore1 += s1
        skore2 += s2
        
        historie1.append(tah1)
        historie2.append(tah2)
       
    return skore1, skore2



### Kódování tabulky reaktivního agenta
Bude se jednat o hledání genomu s nejlepší univerzální stategií, bez specializace na druh strategie protivníka (ať už předem známé, či dedukované v průběhu hry). Turnaj má pevný počet dvaceti kol, paměť reaktivního agenta tak může být také o pevné délce (N = 19, plus stav 0 v prvním kole). 

Nejjednodušší kódování genomu vychází z následujícího mapování čtyř možných situací v každém kole:
- (0, 0) -> 0
- (0, 1) -> 1
- (1, 0) -> 2
- (1, 1) -> 3

Genom v podobě seznamu tedy bude složený z hodnot 0/1 (rozhodnutí v každém kole) seřazených ve směru vývoje hry po čtveřicích podle namapovaných indexů výše. Plus jediné hodnoty prvního kola, ve kterém nelze vycházet z výsledků předchozího kola.

### Evoluce jedince s nejlepší strategií + definice funkce *strategy*
Nejlepší jedinec (genom) je vyvinutý jako univerzální protivník čtyř základních soupeřících strategií. Genom je vypsán jak vcelku v podobě seznamu o délce 1 + 4*(počet kol - 1), pro přehlednost pak i ve čtveřicích po jednotlivých kolech.

In [10]:
# Parametry
GENOME_LENGTH = 1 + 4*19  # 1 + 4*(STEPSNUM - 1)
POPULATION_SIZE = 100
GENERATIONS = 50
TOURNAMENT_SIZE = 3
CROSSOVER_PROBABILITY = 0.7
MUTATION_PROBABILITY = 0.1
STEPSNUM = 20  # počet kol turnaje

# Funkce pro dekódování genomu do strategie - použijeme ji jak při evoluci, tak pro výslednou best_strategy
def decode_strategy(genome):
    def strategy(myhistory, otherhistory):
        if not myhistory:  # První kolo
            return genome[0]
        
        # Další kola
        last_my = myhistory[-1]
        last_other = otherhistory[-1]
        
        # Mapování situace na index v genomu
        situation_index = last_my * 2 + last_other  # 0,0->0; 0,1->1; 1,0->2; 1,1->3
        current_round = len(myhistory)
        genome_index = 1 + situation_index + 4 * (current_round - 1)
        
        # Pokud jsme v posledním kole nebo mimo rozsah genomu
        if current_round >= 19 or genome_index >= len(genome):
            return genome[1 + (situation_index % 4)]
        
        return genome[genome_index]
    
    return strategy

# Vytvoření typů pro DEAP
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Inicializace toolboxu
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=GENOME_LENGTH)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Funkce pro vyhodnocení fitness jedince proti základním strategiím
def evaluate(individual):
    strategy = decode_strategy(individual)
    
    # Seznam strategií, proti kterým budeme hrát
    opponents = [always_cooperate, random_answer, always_betray, tit_for_tat]
    
    total_score = 0
    for opponent in opponents:
        my_score, _ = play(strategy, opponent, STEPSNUM)
        total_score += my_score
    
    return (total_score,)

# Registrace genetických operátorů
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=TOURNAMENT_SIZE)

# Hlavní evoluční smyčka
def run_evolution():
    # Inicializace populace
    pop = toolbox.population(n=POPULATION_SIZE)
    
    # Nastavení statistik
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    # Hall of Fame pro uchování nejlepšího jedince
    hof = tools.HallOfFame(1)
    
    # Spuštění evoluce
    pop, logbook = algorithms.eaSimple(
        pop, toolbox, 
        cxpb=CROSSOVER_PROBABILITY, 
        mutpb=MUTATION_PROBABILITY, 
        ngen=GENERATIONS,
        stats=stats, 
        halloffame=hof, 
        verbose=True
    )
    
    # Výpis statistik
    max_fitness = logbook.select("max")
    avg_fitness = logbook.select("avg")
    
    print(f"Nejlepší fitness: {max_fitness[-1]}")
    print(f"Průměrná fitness poslední generace: {avg_fitness[-1]}")
    
    # Výpis nejlepšího genomu
    best_genome = hof[0]
    print(f"Nejlepší genom: {best_genome}")

    print(f"Výpis nejlepšího genomu po kolech:")
    print(f"[{best_genome[0]}]")
    for i in range(1, len(best_genome), 4):
        ctverice = best_genome[i : i + 4]
        print(ctverice)
    
    # Test nejlepšího genomu proti všem základním strategiím
    """ best_strategy = decode_strategy(best_genome)
    for opponent_func in [always_cooperate, random_answer, always_betray, tit_for_tat]:
        score1, score2 = play(best_strategy, opponent_func, STEPSNUM)
        print(f"Proti {opponent_func.__name__}: {score1} : {score2}") """
    
    # Globální proměnná pro použití v best_strategy
    global BEST_GENOME
    BEST_GENOME = best_genome.copy()
    
    return best_genome

# Spuštění evoluce
BEST_GENOME = run_evolution()

d:\data\docs\unicorn\kurzy\AI\python\.venv\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
d:\data\docs\unicorn\kurzy\AI\python\.venv\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg   	min	max
0  	100   	181.72	161	200
1  	70    	184.75	150	205
2  	65    	187.04	158	207
3  	83    	187.95	168	214
4  	71    	190.33	167	214
5  	85    	191.42	164	217
6  	84    	189.85	160	214
7  	77    	190.78	166	215
8  	61    	193.68	173	215
9  	73    	195.34	166	215
10 	79    	195.83	168	225
11 	69    	199.34	177	225
12 	78    	196.99	178	226
13 	84    	197.26	173	215
14 	65    	199.78	174	218
15 	84    	199.91	175	231
16 	76    	200.69	172	231
17 	68    	202.68	178	231
18 	68    	202.14	177	231
19 	60    	204.53	175	231
20 	70    	203.21	175	231
21 	74    	202.1 	165	231
22 	62    	205.67	175	232
23 	83    	202.57	177	232
24 	70    	204.12	176	232
25 	82    	203.54	182	227
26 	83    	206.22	184	232
27 	72    	205.03	180	232
28 	69    	205.77	175	232
29 	72    	208.16	183	232
30 	56    	210.55	169	232
31 	76    	206.68	172	232
32 	82    	205.36	180	232
33 	71    	207.47	181	232
34 	79    	206.62	178	237
35 	73    	208.59	178	231
36 	75    	209.18	186	231
37 	74    	2

### Nejlepší hráčská strategie vygenerovaná za pomoci DEAP
 Funkce *best_strategy* (a.k.a. *zrada*), která bude v následujícím turnaji soutěžit s ostatními strategiemi, využívá obsah funkce *strategy* implementované v rámci evoluce optimálního genomu o blok výš. 
 
 Pozn.: Funkce je v zadání označené jako *zrada*. Není úplně jasné, vyjma volby formulované jako zradit ano/ne, čím by se měla lišit oproti nejlepší evolučně vygenerované strategii, a zároveň může být matoucí podobnost názvu s *always_betray*, proto jsem zvolila snad příhodnější pojmenování *best_strategy*.

In [11]:
def best_strategy(myhistory, otherhistory):
    
    genome = BEST_GENOME  # if 'BEST_GENOME' in globals() else default_genome  # default_genome = [0] + [0, 1, 1, 0] * 19
    
    strategy_func = decode_strategy(genome)
    return strategy_func(myhistory, otherhistory)

## Turnaj

In [ ]:
ucastnici = [always_cooperate, random_answer, always_betray, tit_for_tat, best_strategy]

#STEPSNUM = 20  # definováno jako parametr evoluce

l = len(ucastnici)
skores = [0 for i in range(l)]

print("=========================================")
print("Turnaj")
print("hra délky:", STEPSNUM)
print("-----------------------------------------")


for i in range(l):
    for j in range(i+1, l):
        f1 = ucastnici[i]
        f2 = ucastnici[j]
        skore1, skore2 = play(f1, f2, STEPSNUM)
        print(f1.__name__, "x", f2.__name__, " ", skore1, ":", skore2)
        skores[i] += skore1
        skores[j] += skore2
        

print("=========================================")
print("= Výsledné pořadí")
print("-----------------------------------------")


# setrideni indexu vysledku
index = sorted(range(l), key=lambda k: skores[k], reverse=True)  # UPRAVENO sestupné řazení

poradi = 1
for i in index:
    f = ucastnici[i]
    print(poradi, ".", f.__name__, ":", skores[i])
    poradi += 1
        
        

Turnaj
hra délky: 20
-----------------------------------------
always_cooperate x random_answer   24 : 84
always_cooperate x always_betray   0 : 100
always_cooperate x tit_for_tat   60 : 60
always_cooperate x best_strategy   6 : 96
random_answer x always_betray   7 : 72
random_answer x tit_for_tat   43 : 43
random_answer x best_strategy   34 : 59
always_betray x tit_for_tat   24 : 19
always_betray x best_strategy   44 : 14
tit_for_tat x best_strategy   49 : 54
= Výsledné pořadí
-----------------------------------------
1 . always_betray : 240
2 . best_strategy : 223
3 . tit_for_tat : 171
4 . random_answer : 168
5 . always_cooperate : 90


## Závěr
Na první pohled možná překvapivě není evolučně generovaná strategie celkovým vítězem turnaje každý proti každému. Je jí "vždy zraď", čemuž však nahrává koncept vězňova dilematu, který zradu obecně zvýhodňuje, zvlášť při nemožnosti vyjednávat. Možná proto byl v zadání původní název funkce *zrada* :) Při existenci vždy zrazující strategie bez ohledu na okolnosti by mohl evolučně generovaný reaktivní agent v přímém souboji nejlépe remizovat. V turnaji pak vítězit, pokud by se mu dařilo získat víc bodů nad ostatními soupeři (např. otestovat na začátku jejich typ). 

Zajímavějším aspektem vězňova dilematu je ovšem vzájemná interaktivita hráčů. Spíš než vyvinutí univerzální strategie vůči primitivním protivníkům by tak mohlo být zajímavější vyvinout dvě vzájemně soupeřící a zároveň ovlivňující se strategie. S možností sledovat podrobněji jejich vzájemné interakce a logiku vývoje vztahu.

Na závěr je třeba dodat, že důležitým parametrem (kromě kódovací tabulky genomu a nastavení evolučního algoritmu) je i tabulka odměn. Aktuálně je využitá tzv. Axelrodova matice, nejběžnější bodovací systém zachovávající dvě základní podmínky: "T (temptation) > R (reward) > P (punishment) > S (sucker)" a "2R > T + S".